In [1]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import  pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import string
import time
import re
trim = re.compile(r'[^\d.,]+')

In [2]:
headers = {'user-agent' : 'Chrome (Macintosh; Intel Mac OS X 10_12_2); Erik Webb Provo, Utah erikgregorywebb@gmail.com'}

### All General Conference Links

In [3]:
# Scrape the General Conference archive page for links to all sessions
scrape_url = "https://www.lds.org/general-conference/conferences?lang=eng"
r = requests.get(scrape_url)
soup = BeautifulSoup(r.content, "lxml")

# Create a list of links, keeping only those on the page that connect to specific sessions
conf_links = list()
for link in soup.find_all('a'):
    conf_links.append(link.get('href'))
conf_links = conf_links[120:214]

# Add the neccesary text to make it a proper URL
for i in range(0,len(conf_links)):
    conf_links[i] = 'https://www.lds.org' + conf_links[i]
    
conf_links[0:5]

['https://www.lds.org/general-conference/2017/04?lang=eng',
 'https://www.lds.org/general-conference/2017/10?lang=eng',
 'https://www.lds.org/general-conference/2016/04?lang=eng',
 'https://www.lds.org/general-conference/2016/10?lang=eng',
 'https://www.lds.org/general-conference/2015/04?lang=eng']

### All Talk Links

In [4]:
all_talk_links = list()

for i in range(0, len(conf_links)):
    time.sleep(1)
    scrape_url = conf_links[i]
    r = requests.get(scrape_url)
    soup = BeautifulSoup(r.content, "lxml")
    
    links = list()
    for link in soup.find_all('a'):
        links.append(link.get('href'))
        
    # Clean the list
    links_df = pd.DataFrame({'col':links})
    links_df = links_df.dropna(how='any')
    value = conf_links[i][20:46] + "/"
    good_links = links_df['col'].str.contains(value)   
    links_df = links_df[good_links]
    
    talk_links_list = list(links_df['col'])
    for i in range(0,len(talk_links_list)):
        talk_links_list[i] = 'https://www.lds.org' + talk_links_list[i]
    all_talk_links.extend(talk_links_list)

In [5]:
len(all_talk_links)

3699

In [6]:
all_talk_links[0]

'https://www.lds.org/general-conference/2017/04/trust-in-the-lord-and-lean-not?lang=eng'

### Extract Individual Talk Data

In [7]:
# Define function
def scrape_element(css_selector):
    return (page.select_one(css_selector)).get_text()

In [11]:
gf_data = list()
for i in range(0, 5):
    time.sleep(1)
    scrape_url = all_talk_links[i]
    r = requests.get(scrape_url)
    page = BeautifulSoup(r.content, "lxml")
    try:
        title = scrape_element('.title div')
        speaker = scrape_element('.article-author__name')
        calling = scrape_element('.article-author__title')
        text = scrape_element('.body-block')
        talk = list([title, speaker, calling, text])
        gf_data.append(talk)
    except Exception:
        print("Error: ", i)

Error:  2


In [12]:
gf_df = pd.DataFrame(gf_data)
gf_df.head()

,0,1,2,3
0,Trust in the Lord and Lean Not,By Bonnie H. Cordon,Second Counselor in the Primary General Presid...,"While I was traveling in Asia, a dear sister a..."
1,The Beauty of Holiness,By Carol F. McConkie,First Counselor in the Young Women General Pre...,"As I have prepared for this meeting, my heart ..."
2,“My Peace I Leave with You”,By President Henry B. Eyring,First Counselor in the First Presidency,"My dear sisters, we have been blessed by the S..."
3,Gathering the Family of God,By President Henry B. Eyring,First Counselor in the First Presidency,"My dear brothers and sisters, I rejoice at the..."


### Write Data to CSV

In [10]:
writer_orig = pd.ExcelWriter('GF_DATA.xlsx', engine='xlsxwriter')
gf_df.to_excel(writer_orig, index=False, sheet_name='GF_DATA')
writer_orig.save()

### NLP/Text Mining

See next script.